<a href="https://colab.research.google.com/github/jjangmo91/Cervus-nippon/blob/main/Cervus_nippon_prep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. 환경 설정 및 라이브러리 임포트

In [ ]:
# 필수 패키지 설치
!pip install eesdm -q
!pip install geojson -q
!pip install geemap -U -q

# 라이브러리 임포트
import ee
import geemap
import pandas as pd
import eeSDM
from ipyleaflet import WidgetControl
from ipywidgets import Label
import glob

# Earth Engine 인증 및 초기화
ee.Authenticate()
ee.Initialize(project='ee-jjangmo91')

# Google Drive 마운트
from google.colab import drive
drive.mount('/content/drive')
data_path = '/content/drive/MyDrive/KNPS/Deer/Ecotopia_Data_2024_2025/'

# 2. GPS 원본 데이터 전처리 및 분석용 CSV 파일 저장

In [ ]:
# 개체별 GPS 파일 목록 불러오기
dsf_files = sorted(glob.glob(data_path + 'DSF-*.csv'))
dsm_files = sorted(glob.glob(data_path + 'DSM-*.csv'))
print(f"암컷(DSF) 파일 {len(dsf_files)}개, 수컷(DSM) 파일 {len(dsm_files)}개를 발견했습니다.")

# 데이터 통합 함수 정의
def combine_gps_data(file_list):
    df_list = [pd.read_csv(file) for file in file_list]
    return pd.concat(df_list, ignore_index=True)

# 성별로 데이터 통합
df_female = combine_gps_data(dsf_files)
df_male = combine_gps_data(dsm_files)
df_all = pd.concat([df_female, df_male], ignore_index=True)

# 시간 정보(Collecting_time)를 날짜/시간 형태로 변환
for df in [df_female, df_male, df_all]:
    df['Collecting_time'] = pd.to_datetime(df['Collecting_time'], errors='coerce')

# 분석 그룹(전체, 암컷, 수컷)별로 반복 처리
datasets_to_create = {'all': df_all, 'female': df_female, 'male': df_male}

for name, df in datasets_to_create.items():
    # 전체 기간 데이터셋 저장
    sdm_entire = pd.DataFrame({
        'longitude': df['Longitude'],
        'latitude': df['Latitude'],
        'cervus': 'cervus-nippon'
    })
    output_filename_entire = f'sdm_occurrences_{name}_entire_raw.csv'
    sdm_entire.to_csv(data_path + output_filename_entire, index=False)
    print(f"  - [{name.upper()} 전체 기간] {len(sdm_entire):,}개 좌표 저장 완료.")

    # 겨울철(12월~2월) 데이터셋 저장
    df_winter = df[df['Collecting_time'].dt.month.isin([12, 1, 2])].copy()
    if not df_winter.empty:
        sdm_winter = pd.DataFrame({
            'longitude': df_winter['Longitude'],
            'latitude': df_winter['Latitude'],
            'cervus': 'cervus-nippon'
        })
        output_filename_winter = f'sdm_occurrences_{name}_winter_raw.csv'
        sdm_winter.to_csv(data_path + output_filename_winter, index=False)
        print(f"  - [{name.upper()} 겨울철] {len(sdm_winter):,}개 좌표 저장 완료.")

#3. 전처리된 4개 그룹 데이터 시각화

In [ ]:
# 시각화용 데이터 및 연구지역(AOI) 불러오기
df_female_entire = pd.read_csv(data_path + 'sdm_occurrences_female_entire_raw.csv')
df_male_entire = pd.read_csv(data_path + 'sdm_occurrences_male_entire_raw.csv')
df_female_winter = pd.read_csv(data_path + 'sdm_occurrences_female_winter_raw.csv')
df_male_winter = pd.read_csv(data_path + 'sdm_occurrences_male_winter_raw.csv')
print("\nVisualization data for 4 groups loaded successfully.")

# 속리산 국립공원 경계(AOI) 불러오기 (WDPA ID: 773)
protected_areas = ee.FeatureCollection("WCMC/WDPA/current/polygons")
aoi = protected_areas.filter(ee.Filter.eq('WDPAID', 773)).geometry()
print("Songnisan National Park AOI loaded.")
print("-" * 30)

# geemap 호환성을 위해 모든 데이터프레임의 열 이름을 변경
for df in [df_female_entire, df_male_entire, df_female_winter, df_male_winter]:
    df.rename(columns={'x': 'longitude', 'y': 'latitude'}, inplace=True)

# pandas 데이터프레임을 ee.FeatureCollection으로 변환
ee_f_entire = geemap.pandas_to_ee(df_female_entire)
ee_m_entire = geemap.pandas_to_ee(df_male_entire)
ee_f_winter = geemap.pandas_to_ee(df_female_winter)
ee_m_winter = geemap.pandas_to_ee(df_male_winter)

# geemap을 이용한 대화형 지도 생성
Map = geemap.Map(center=[36.54, 127.85], zoom=11)

# 지도에 레이어 추가 (겨울철 데이터를 더 잘 보이게 위에 추가)
Map.addLayer(aoi, {'color': '#006600', 'fillColor': '#33996655'}, 'Songnisan_NP_AOI')
Map.addLayer(ee_f_entire, {'color': '#F08080'}, 'Female (Entire)') # 암컷 전체: 연한 분홍
Map.addLayer(ee_m_entire, {'color': '#87CEEB'}, 'Male (Entire)')   # 수컷 전체: 연한 파랑
Map.addLayer(ee_f_winter, {'color': '#DC143C'}, 'Female (Winter)') # 암컷 겨울: 진한 분홍
Map.addLayer(ee_m_winter, {'color': '#0000CD'}, 'Male (Winter)')   # 수컷 겨울: 진한 파랑

# 범례(Legend) 추가
Map.add_legend(
    title="Legend",
    legend_dict={
        "Female (Entire Period)": "F08080",
        "Male (Entire Period)": "87CEEB",
        "Female (Winter)": "DC143C",
        "Male (Winter)": "0000CD"
    }
)

# 제목 추가
title_widget = Label(value="Sika Deer Occurrence by Sex and Season")
Map.add_control(WidgetControl(widget=title_widget, position='topright'))

# 지도 출력
display(Map)